In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import kagglehub
snap_amazon_fine_food_reviews_path = kagglehub.dataset_download('snap/amazon-fine-food-reviews')

print('Data source import complete.')


Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/Reviews.csv
/kaggle/input/amazon-fine-food-reviews/database.sqlite


In [3]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import torch
from torch import nn
from torch.nn import functional as F
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np


2025-06-28 19:39:31.883322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751139572.104647      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751139572.170809      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [5]:
#preparing the data ready for the  transformer
data = pd.read_csv('/kaggle/input/amazon-fine-food-reviews/Reviews.csv')


In [6]:
data_train = data.iloc[:3000, :]

In [7]:

#preparing the inputs and the outputs from the data

inputs = [];
text="";

output = [];
text2="";
for row in np.array(data_train.iloc[:, :]):
    if len(row[-1])> 300:
        continue;

    inputs.append(row[-1])
    text = text+ row[-1];
    output.append(row[-2])
    text2 = text2+ row[-2];







In [8]:
output[0]

'Good Quality Dog Food'

In [9]:
data_train.iloc[20, :][-1]

/tmp/ipykernel_13/3301232396.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_train.iloc[20, :][-1]


"My husband is a Twizzlers addict.  We've bought these many times from Amazon because we're government employees living overseas and can't get them in the country we are assigned to.  They've always been fresh and tasty, packed well and arrive in a timely manner."

In [10]:
print(max([len(x) for x in inputs]))
print(max([len(x) for x in output]))

300
98


In [11]:
#tokenizer to fit the input tokens
tokenizer = Tokenizer();

X = [];

tokenizer.fit_on_texts(inputs);
sequences = tokenizer.texts_to_sequences(inputs);
for sequence in sequences:
    X.append(sequence);



In [12]:
#tokenizer to fit the output tokens
token_out = Tokenizer();
Y = [];
token_out.fit_on_texts(output);
sequences = token_out.texts_to_sequences(output);
for sequence in sequences:
    Y.append(sequence);

In [13]:
max([max(i) for i in Y])

1397

In [14]:
maxx = min( [min(i) for i in Y] )

In [15]:
maxx

1

In [16]:
len(token_out.word_counts)

1397

In [17]:
#the output has to be prepared according to forced teacher training, so the inputs and targets are being
#prepared accordingly
def prepare(tokenized_summaries, start_token=1398, end_token=1399):
    Y_input = []
    Y_target = []
    for seq in tokenized_summaries:
        Y_input.append([start_token] + seq)
        Y_target.append(seq + [end_token])
    return Y_input, Y_target

In [18]:
y_input , y_output = prepare(Y);

In [19]:
#paddinf all the tensors, finally getting the data ready for the model
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_padded = pad_sequences(X,maxlen=302, padding='post')          # encoder input
Y_input_padded = pad_sequences(y_input, maxlen=102, padding='post')   # decoder input
Y_target_padded = pad_sequences(y_output, maxlen=102, padding='post') # decoder target

TRANSFORMER IMPLEMENTATION FUNCTIONS


In [20]:
#Self attention Block
dropout = 0.2
n_embd = 32
# block_size = 302
batch_size = 10
head_size = 4
vocab_size_e = 6000
vocab_size_d = 6000
# n_heads = 2
import torch
import torch.nn as nn
import torch.nn.functional as F

class Head(nn.Module):
    def __init__(self, block_size):
        super().__init__()
        self.key   = nn.Linear(n_embd , head_size, bias=False)
        self.query = nn.Linear(n_embd , head_size, bias=False)
        self.value = nn.Linear(n_embd , head_size, bias=False)

        self.tril = torch.tril(torch.ones(block_size, block_size))  # (T, T)
        self.dropout = nn.Dropout(0.1)

    def forward(self, X):
        B, T, C = X.size()

        assert T <= self.tril.size(0), f"Sequence length {T} exceeds tril size {self.tril.size(0)}"

        # Step 1: Compute key, query, value
        k = self.key(X)      # (B, T, head_size)
        q = self.query(X)    # (B, T, head_size)

        # Step 2: Compute attention scores
        wei = q @ k.transpose(-2, -1) * (k.shape[-1] ** -0.5)  # (B, T, T)

        # Step 3: Causal masking
        mask = self.tril[:T, :T] == 0
        wei = wei.masked_fill(mask.unsqueeze(0).to(X.device), float('-inf'))

        # Step 4: Softmax to get weights
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        # Step 5: Weighted sum of values
        v = self.value(X)        # (B, T, head_size)
        out = wei @ v            # (B, T, head_size)

        return out


  




In [21]:
class feedForward(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(n_embd, 4 * n_embd)
    self.relu = nn.ReLU();
    self.layer_f = nn.Linear(4*n_embd, n_embd);

  def forward(self, x):
      x = self.layer1(x);
      x = self.relu(x);
      x = self.layer_f(x);
      return x;


In [22]:
class MultiHeadAttention(nn.Module):


    def __init__(self, block_size):
        super().__init__()

        self.heads = nn.ModuleList([Head(block_size) for _ in range(8)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out






In [23]:
class decoder_Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = MultiHeadAttention(102)
        self.ffwd = feedForward()
        self.ln1 = nn.LayerNorm()
        self.ln2 = nn.LayerNorm()

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [24]:
class encoder_block(nn.Module):
  def __init__(self):
    super().__init__()

    self.sa = MultiHeadAttention(302)
    self.ffwd = feedForward()
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
     x = x + self.sa(self.ln1(x))
     x = x + self.ffwd(self.ln2(x))
     return x


In [25]:
class crossAttention(nn.Module):
  def __init__(self):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias = False)
    self.l_f = nn.Linear(head_size, n_embd, bias = False)


    self.dropout = nn.Dropout(dropout);

  def forward(self, X, Y):
     k = self.key(X);
     q = self.query(Y);

     wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5

     wei = F.softmax(wei, dim=-1)
     v = self.value(X);
     out =  wei@v;
     return self.l_f(out);





In [26]:
class decoder_Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()

        self.sa = MultiHeadAttention(102)
        self.ffwd = feedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.cross =  crossAttention();
        self.ln2 = nn.LayerNorm(n_embd)


    def forward(self, x, X_enc):
        x = x + self.sa(self.ln1(x))
        x = x+ self.cross(X_enc, x);
        x = x + self.ffwd(self.ln2(x))
        return x



In [27]:

class Encoder(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(6000, n_embd, padding_idx =0)
        self.position_embedding_table = nn.Embedding(302, n_embd)
        self.blocks = nn.Sequential(*[encoder_block() for _ in range(2)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm


    def forward(self, idx, targets=None):
        B, T = idx.shape
        #10000
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)##
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        return x


In [28]:
class Decoder(nn.Module):

    def __init__(self):
        super().__init__()
       
        self.token_embedding_table = nn.Embedding(6000, n_embd, padding_idx =0)
        self.position_embedding_table = nn.Embedding(102, n_embd)
        self.blocks = nn.Sequential(*[decoder_Block() for _ in range(2)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, 6000)

    def forward(self, idx,x_enc, targets=None):
        B, T = idx.shape


        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)####
        x = tok_emb + pos_emb # (B,T,C)
        for block in self.blocks:

          x = block(x, x_enc)

        x = self.ln_f(x) # (B,T,C)
        return self.lm_head(x)


In [29]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = Encoder();
    self.decoder = Decoder();

  def forward(self, X,y_in, y_target, loss_f):
    x_enc = self.encoder(X);
    y_out = self.decoder(y_in, x_enc);
    loss = loss_f(y_out.view(-1, y_out.shape[-1]), y_target.view(-1))
    return y_out, loss;

In [30]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Move all data to tensors first (CPU for now)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_tensor  =   torch.tensor(X_padded, dtype=torch.long).to(device)
y_in_tensor = torch.tensor(Y_input_padded, dtype=torch.long).to(device)
y_tgt_tensor = torch.tensor(Y_target_padded, dtype=torch.long).to(device)

# Create dataset and dataloader
dataset = TensorDataset(x_tensor, y_in_tensor, y_tgt_tensor)
batch_size = 4
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Device setup


# Initialize model and optimizer
model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_f = nn.CrossEntropyLoss()
import os

# Directory to save checkpoints
os.makedirs("checkpoints", exist_ok=True)

# Training loop
for epoch in range(10):
    model.train()
    total_loss = 0.0

    for x_batch, y_in_batch, y_tgt_batch in loader:
        # Move batch to device
        x_batch = x_batch.to(device)
        y_in_batch = y_in_batch.to(device)
        y_tgt_batch = y_tgt_batch.to(device)

        # Forward pass and loss
        y_out, loss_val = model(x_batch, y_in_batch, y_tgt_batch, loss_f)

        # Backpropagation
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        total_loss += loss_val.item()
        if epoch==0:
            print(total_loss)

    avg_loss = total_loss / len(loader)
    
    if epoch % 1 == 0:
        print(f"Epoch {epoch} - Avg Loss: {avg_loss:.4f}")
        
        # Save the model
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, f"checkpoints/model_epoch_{epoch}.pt")






9.060674667358398
17.76961612701416
26.132694244384766
34.184327125549316
41.92098426818848
49.40656280517578
56.64831781387329
63.61801815032959
70.38274574279785
76.98796033859253
83.39785623550415
89.65489673614502
95.7546558380127
101.73001718521118
107.5423583984375
113.32670211791992
118.9329628944397
124.4528341293335
129.88997650146484
135.16114139556885
140.42226600646973
145.56894063949585
150.61415338516235
155.61022901535034
160.49369621276855
165.2644443511963
169.97202348709106
174.55251216888428
179.11578035354614
183.61120223999023
188.00010108947754
192.28986883163452
196.48512935638428
200.65358209609985
204.71244955062866
208.6753752231598
212.51839184761047
216.23669719696045
219.9153654575348
223.60316348075867
227.15614438056946
230.71815872192383
234.13365364074707
237.50424575805664
240.77190732955933
243.9594919681549
246.99234080314636
249.95424056053162
252.85032629966736
255.6084725856781
258.35345220565796
261.1009736061096
263.7110540866852
266.35107493400

In [31]:
model = Model().to(device)                 # same architecture as before
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  # same optimizer config
checkpoint = torch.load("/kaggle/working/checkpoints/model_epoch_9.pt")

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

epoch = checkpoint['epoch']
loss = checkpoint['loss']

print(f"Loaded model from epoch {epoch} with loss {loss:.4f}")

Loaded model from epoch 9 with loss 0.1832


In [32]:
x_tensor = torch.tensor(X_padded, dtype=torch.long, device="cpu").to(device)

In [33]:
import os

files = os.listdir("/kaggle/working/checkpoints")
print(files)


['model_epoch_2.pt', 'model_epoch_7.pt', 'model_epoch_1.pt', 'model_epoch_9.pt', 'model_epoch_6.pt', 'model_epoch_8.pt', 'model_epoch_4.pt', 'model_epoch_0.pt', 'model_epoch_3.pt', 'model_epoch_5.pt']


In [34]:
import zipfile
import os

folder_path = "/kaggle/working/checkpoints"
zip_path = "/kaggle/working/checkpoint.zip"

# Create a zip archive of the folder
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, folder_path)
            zipf.write(file_path, arcname)

print("Zipping complete:", zip_path)


Zipping complete: /kaggle/working/checkpoint.zip


In [35]:
import torch
import torch.nn.functional as F
import numpy as np


    # Step 3: Initialize decoder input
def translate(model, tokenizer, sentence, start_token, end_token, device='cpu', max_len=100):
    
    # Tokenize input
    X_token = tokenizer.texts_to_sequences([sentence])
    X_tensor = torch.tensor(X_token, dtype=torch.long).to(device)

    # Encode
    encoder_output = model.encoder(X_tensor)

    # Initialize decoder
    decoder_input = torch.tensor([[start_token]], dtype=torch.long).to(device)
    output_tokens = []

    for _ in range(max_len):
        # Decode
        logits = model.decoder(decoder_input, encoder_output)  # (1, seq_len, vocab_size)
        probs = F.softmax(logits[:, -1, :], dim=-1)             # only last token

        # Sample next token
        next_token = torch.argmax(probs, dim=-1).item()


        if next_token == end_token:
            break

        output_tokens.append(next_token)

        # Append next token to decoder input
        decoder_input = torch.cat(
            [decoder_input, torch.tensor([[next_token]], dtype=torch.long).to(device)],
            dim=1
        )

    return output_tokens

    



In [36]:
sen = "  We've bought these many times from Amazon because we're government employees living overseas and can't get them in the country we are assigned to.  They've always been fresh and tasty, packed well and arrive in a timely manner"
output_ids = translate(model, tokenizer,sen, 1398, 1399)
translated_text = tokenizer.sequences_to_texts([output_ids])
print("Translated:", translated_text)


Translated: ['the you']
